In [1]:
import math
import ipywidgets
import time

def get_opt_prc_one_thd(opt_type, strike_prc, spot_prc, remain_day, interest_rate, volatility, node_cnt):
    st_time = time.time()
    
    remain_year = remain_day / 365
    delta_t = max(remain_year / node_cnt, 0.000001)
    a = math.exp(interest_rate * 0.01 * delta_t)
    u = math.exp(volatility * 0.01 * math.sqrt(delta_t))
    d = 1 / u
    p = (a - d) / (u - d)
    r_p = 1 - p

    datas = [0] * (node_cnt + 1)

    for row in range(node_cnt+1):
        datas[row] = spot_prc * (u**(node_cnt-row)) * (d**row)
        #datas[row] = max(datas[row]-strike_prc, 0) if "call" == opt_type else max(strike_prc-datas[row], 0)
        
    print(datas)

    for col in range(node_cnt, 0, -1):
        for row in range(col):
            datas[row] = (p * datas[row] + r_p * datas[row+1]) / a
    
    ed_time = time.time()
    
    print("옵션가격:", datas[0])
    print("소요시간(초):", ed_time - st_time)

ipywidgets.interact_manual(
    get_opt_prc_one_thd,
    opt_type=ipywidgets.Dropdown(options=[("콜", "call"), ("풋", "put")], description="옵션타입"),
    spot_prc=ipywidgets.BoundedFloatText(min=0, max=1000, value=274.34, step=0.01, description="기초자산"),
    strike_prc=ipywidgets.BoundedFloatText(min=0, max=1000, value=275, step=2.5, description="행사가"),
    remain_day=ipywidgets.BoundedIntText(min=1, max=1000, value=31, description="잔존일"),
    interest_rate=ipywidgets.BoundedFloatText(min=0, value=1.55, step=0.01, description="이자율(%)"),
    volatility=ipywidgets.BoundedFloatText(min=0.01, value=13.59, step=0.01, description="변동성(%)"),
    node_cnt=ipywidgets.BoundedIntText(min=2, max=100000, value=49, description="노드수"),
);

interactive(children=(Dropdown(description='옵션타입', options=(('콜', 'call'), ('풋', 'put')), value='call'), Bound…

In [3]:
import math
import ipywidgets
import time
import numpy as np
import pycuda.autoinit
import pycuda.driver as drv
from pycuda import compiler

OPT_TYPE_CALL = 0
OPT_TYPE_PUT = 1

# Kernel code
kernel_code = """
__constant__ int opt_type;
__constant__ float params[7];

__global__ void get_opt_prc_n_thd(float* ret)
{
    extern __shared__ float datas[];
    
    int node_cnt = blockDim.x;
    int tid = threadIdx.x;
    
    const float spot_prc = params[0];
    const float strike_prc = params[1];
    const float a = params[2];
    const float u = params[3];
    const float d = params[4];
    const float p = params[5];
    const float r_p = params[6];
    
    //datas[tid] = spot_prc * powf(u, node_cnt-tid) * powf(d, tid);
    //datas[tid] = ( 1 == opt_type ) ? fmaxf(datas[tid]-strike_prc, 0) : fmaxf(strike_prc-datas[tid], 0);
    
    __syncthreads();
    
    //ret[tid] = datas[tid];
    
    ret[tid] = 1.0;
}
"""

# Compile the kernel code
mod = compiler.SourceModule(kernel_code)

# result
#result = np.zeros(1, dtype=np.float)

opt_type = OPT_TYPE_CALL
spot_prc = 274.34
strike_prc = 275.0
remain_day = 31
interest_rate = 1.55
volatility = 13.59
node_cnt = 5

remain_year = remain_day / 365
delta_t = max(remain_year / node_cnt, 0.000001)
a = math.exp(interest_rate * 0.01 * delta_t)
u = math.exp(volatility * 0.01 * math.sqrt(delta_t))
d = 1 / u
p = (a - d) / (u - d)
r_p = 1 - p

params = [spot_prc, strike_prc, a, u, d, p, r_p]

inputs_int = np.array([opt_type], dtype=np.int)
inputs_float = np.array(params, dtype=np.float32)

drv.memcpy_htod(mod.get_global("opt_type")[0], inputs_int)
drv.memcpy_htod(mod.get_global("params")[0], inputs_float)

# Get kernel function
get_opt_func = mod.get_function("get_opt_prc_n_thd")

# Run & Print
N = 5
result_arr = np.zeros(N, dtype=np.float)
get_opt_func(drv.InOut(result_arr), block=(N, 1, 1), shared=N)

print(inputs_float)
print(result_arr)

[274.34       275.           1.0002633    1.0178698    0.98244387
   0.50300515   0.49699485]
[7.81250185e-003 7.81250185e-003 5.26354425e-315 0.00000000e+000
 0.00000000e+000]
